In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from time import time, gmtime, strftime

from bert.tokenization.bert_tokenization import FullTokenizer

from utils.models.modelUtils import loadMeanFromFile, calcAccConcatModel

from final_models import create_image_inceptionv3_model, create_text_model, buildConcatModelTitleComments

from utils.callbacks.MyCallbacks import MyCallbacks
from utils.datagenUtils.dual_modal.DataSeqTitleComments import DataSequenceTitleComments
from utils.datagenUtils.datagenUtils import checkDataframe

from utils.telegramUtils.telegram_bot import telegram_send_message
from utils.textUtils.textpreprocessing import FakeDetectionDataTrainVal, FakeDetectionDataTest
from utils.textUtils.commentsProcessing import FakeDetectionDataCommentsTest, FakeDetectionDataCommentsTrainVal

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from sklearn.metrics import accuracy_score


In [2]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 2048
EPOCHS = 10

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

# Bert Parameters
MAX_SEQUENCE_LENGTH = 128 # from model definition!

# Training switches
RETRAIN_WHOLE_MODEL = False
loadPretrained = False # Attention that model weights path is set !

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}, BERT Max sequence length: {MAX_SEQUENCE_LENGTH}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

START
Batch Size: 2048, Epochs: 10, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08, BERT Max sequence length: 128


In [3]:
pathToRootModelDir = '/home/armin/repos/fkd-model-handling/models/best_models'

pathToBertModelTitle = os.path.join(pathToRootModelDir, 'single_text_title', 'weights-improvement-02-0.88.hdf5')
pathToBertModelComments = os.path.join(pathToRootModelDir, 'single_text_comments', 'weights-improvement-03-0.87.hdf5')

pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

titleCommentsModelWeightsPath = ""

root = '/home/armin/repos/fkd-model-handling/'
bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt") 

In [4]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "model_concat_title_comments"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [5]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

Directory /home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_image_2020-07-16_09:06/tensorboard does not exist, creating it instead!


In [6]:
df_train_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train_['2_way_label'] = df_train_['2_way_label'].apply(lambda x: np.array(x))
df_test_['2_way_label'] = df_test_['2_way_label'].apply(lambda x: np.array(x))
df_val_['2_way_label'] = df_val_['2_way_label'].apply(lambda x: np.array(x))

df_train = df_train_
df_test = df_test_
df_val = df_val_

# df_train = df_train[:15000]
# df_test = df_test[:15000]
# df_val = df_val[:15000]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)


In [7]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))

all_title_data = FakeDetectionDataTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

all_title_test = FakeDetectionDataTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)


comments_data_train = FakeDetectionDataCommentsTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

comments_data_test = FakeDetectionDataCommentsTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)



train_title_x = all_title_data.train_x
train_title_val_x = all_title_data.val_x
test_title_x = all_title_test.test_x

comments_train_x = comments_data_train.train_x
comments_val_x = comments_data_train.val_x
comments_test_x = comments_data_test.test_x

559104it [02:06, 4425.89it/s]
57344it [00:13, 4374.63it/s]


max seq_len 687


57344it [00:12, 4500.97it/s]


max seq_len 128


559104it [31:36, 294.88it/s]
57344it [03:22, 282.95it/s]


max seq_len 55475


57344it [03:20, 286.00it/s]


max seq_len 39424


In [8]:
train_seq = DataSequenceTitleComments(df_train, train_title_x, comments_train_x, GLOBAL_BATCH_SIZE)

test_seq = DataSequenceTitleComments(df_test,  test_title_x, comments_test_x, GLOBAL_BATCH_SIZE)

val_seq = DataSequenceTitleComments(df_val,  train_title_val_x, comments_val_x, GLOBAL_BATCH_SIZE)

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE

In [9]:
# def buildConcatModelTitleComments(model_bert_title, model_bert_comments, NUM_CLASS):
#     model_bert_title = model_bert_title.get_layer('bert_output_layer_768').output
#     model_bert_comments = model_bert.get_layer('bert_output_layer_768').output
    
#     # Build new models, without softmax
#     model_bert_title = Model(model_bert_title.inputs, model_bert_title)    
#     model_bert_comments = Model(model_bert_comments.inputs, model_bert_comments)
    
#     concatenate = Concatenate()([model_bert_title.output, model_bert_comments.output]) # Fusion
#     x = Dense(512, activation = 'relu')(concatenate)
#     x = Dropout(0.4)(x)
#     output = Dense(NUM_CLASS, activation='softmax', name='final_multimodal_output')(x)
#     model_concat = Model([model_bert_title.input, model_bert_comments.input], output)
#     return model_concat

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Preparing:
    model_bert_title, model_bert_title_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelTitle, isTrainable=RETRAIN_WHOLE_MODEL)
    model_bert_comments, model_bert_comments_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelComments, isTrainable=RETRAIN_WHOLE_MODEL) 
    
    # Handling same layer name error:
    for i, layer in enumerate(model_bert_title.layers):
        layer._name = layer._name + '_title' # Consider the _ for the setter
    
    model_concat = buildConcatModelTitleComments(model_bert_title, model_bert_comments, 2)
    
    if loadPretrained:
        model_concat.load_weights(titleCommentsModelWeightsPath)

    if verbose:
        print('--------------Text Title MODEL --------------')
        model_bert_title.summary()
        print('--------------Text Comments MODEL --------------')
        model_bert_comments.summary()
        print('--------------CONCAT MODEL --------------')
        model_concat.summary()
        for l in model_concat.layers:
            print(l.name, l.trainable)
    
    
    model_concat.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
          optimizer=optimizer,
          metrics=['accuracy'])
    
    history = model_concat.fit(train_seq,
                    steps_per_epoch=STEP_SIZE_TRAIN,epochs=EPOCHS, 
                    validation_data=val_seq,
                    validation_steps=STEP_SIZE_VAL, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    verbose=TF_VERBOSE,
                    workers=12
                   )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
bert shape (None, 128, 768)
bert shape (None, 128, 768)
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 4 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 4 all-red

In [11]:
# for i, layer in enumerate(model_bert_title.layers):
#     layer._name = layer._name + '_title' # Consider the _ for the setter
        
# for i, layer in enumerate(model_bert_title.layers):
#     print(layer.name)

In [12]:
%%capture

plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained.png'), show_shapes=True, dpi=600, expand_nested=False)
plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained_nested.png'), show_shapes=True, dpi=600, expand_nested=True)

In [13]:
# # For later model loading
# modelFile = '/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_image_2020-07-15_13:16/weights-improvement-02-0.91.hdf5'

# model_concat.load_weights(modelFile)

In [16]:
plotTimesPerEpoch(callbacks_list)

It took so long to train on one epoch: [41, 39, 39, 39, 39, 39, 39, 39, 39, 39] minutes


In [14]:
calcAccConcatModel(model_concat, val_seq, GLOBAL_BATCH_SIZE, 'Val')
calcAccConcatModel(model_concat, test_seq, GLOBAL_BATCH_SIZE, 'Test')

28/28 [==============================] - 189s 7s/step
Val Accuracy is 0.8559047154017857
28/28 [==============================] - 189s 7s/step
Test Accuracy is 0.8571602957589286


In [15]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
